문제를 코드로 번역하는 task로 생각하여  
SentencePiece와 Transformer를 활용하여 학습시켜봤습니다.

pretrain 모델은 사용하지 않고 주어진 데이터를 정제하여 학습시켰습니다.  

전처리는 코드의 \n 을 NN으로 바꿔주는 것만 진행하였고,  

test.csv 파일에 대해 code와 answer를 생성해 json 파일을 만드는 과정까지 나타나 있습니다.

# 1- 필요 라이브러리 호출 및 설치

In [ ]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

import time
import re
import io
import os
import random

#import seaborn # Attention 시각화

In [ ]:
# 폰트 관련 설치 코드인데 시각화를 안 해서 안 써도 됩니다.
'''
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv 
!rm ~/.cache/matplotlib -rf
'''

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent

In [ ]:
'''
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

%config InlineBackend.figure_format = 'retina'

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager.findfont(font)
'''

'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

### SentencePiece 다운로드

In [ ]:
# SentencePiece 다운로드
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2- 데이터 처리하기

### 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# KMWP github가 저장될 경로로 이동
%cd '/content/drive/MyDrive/Ko-Co'

In [ ]:
# KMWP github 복제하기
!git clone https://github.com/tunib-ai/KMWP.git

In [ ]:
# 데이터 불러오기
data_dir = '/content/drive/MyDrive/KMWP/code/data/new_train.csv'

import pandas as pd 

data = pd.read_csv(data_dir,)
data.head()

,index,class,problem,code,answer,Unnamed: 5
0,0,8,한 변의 길이가 24cm인 정육각형과 둘레가 같은 정팔각형이 있습니다. 이 정팔각형...,a = 24\nb = 6\nc = 8\ny = a * b // c\nprint(y),18,NaN
1,1,1,윤아는 부추전을 똑같이 8조각으로 나누어 한 조각을 먹었습니다. 윤미는 같은 크기의...,a = 8\nb = 16\ny = b // a\nprint(y),2,NaN
2,2,8,"화단 주변에 한 변이 12m인 정팔각형 모양의 울타리를 두른다면, 울타리는 모두 몇...",a = 12\nb = 8\ny = a * b\nprint(y),96,NaN
3,3,3,"6장의 숫자 카드 0, 9, 8, 7, 2, 1가 있습니다. 이를, 한 번씩 사용하...",from itertools import permutations\n\na = ['0'...,987210,NaN
4,4,1,0.26 x 0.8을 계산해 주세요.,"a = 0.26\nb = 0.8\ny = a * b\nprint(""{:.2f}"".f...",0.21,NaN


> 전처리한 데이터를 사용했기 때문에 필요 없는 컬럼들은 제거해야 합니다

In [ ]:
# 원래 데이터에 없던 index와 Unnamed 컬럼 제거
data.drop(labels=['index', 'Unnamed: 5'], axis=1, inplace=True) 

In [ ]:
data.head() 

,class,problem,code,answer
0,8,한 변의 길이가 24cm인 정육각형과 둘레가 같은 정팔각형이 있습니다. 이 정팔각형...,a = 24\nb = 6\nc = 8\ny = a * b // c\nprint(y),18
1,1,윤아는 부추전을 똑같이 8조각으로 나누어 한 조각을 먹었습니다. 윤미는 같은 크기의...,a = 8\nb = 16\ny = b // a\nprint(y),2
2,8,"화단 주변에 한 변이 12m인 정팔각형 모양의 울타리를 두른다면, 울타리는 모두 몇...",a = 12\nb = 8\ny = a * b\nprint(y),96
3,3,"6장의 숫자 카드 0, 9, 8, 7, 2, 1가 있습니다. 이를, 한 번씩 사용하...",from itertools import permutations\n\na = ['0'...,987210
4,1,0.26 x 0.8을 계산해 주세요.,"a = 0.26\nb = 0.8\ny = a * b\nprint(""{:.2f}"".f...",0.21


In [ ]:
# problem과 code 리스트화
raw_prob = data.problem.to_list()
raw_code = data.code.to_list()

print(raw_prob[:3])
print(raw_code[:3])

['한 변의 길이가 24cm인 정육각형과 둘레가 같은 정팔각형이 있습니다. 이 정팔각형의 한 변의 길이는 몇 cm인지 구하시오.', '윤아는 부추전을 똑같이 8조각으로 나누어 한 조각을 먹었습니다. 윤미는 같은 크기의 부추전을 똑같이 16 조각으로 나누었습니다. 윤아와 같은 양을 먹으려면 윤미는 몇 조각을 먹어야 할까요?', '화단 주변에 한 변이 12m인 정팔각형 모양의 울타리를 두른다면, 울타리는 모두 몇 m일까요?']
['a = 24\nb = 6\nc = 8\ny = a * b // c\nprint(y)', 'a = 8\nb = 16\ny = b // a\nprint(y)', 'a = 12\nb = 8\ny = a * b\nprint(y)']


### 정제작업
- 코드의 \n 에 대해 NN으로 바꿔줍니다.
  - \n을 그대로 넣었을 때, 코드 생성 시 \n이 나타나지 않는 이슈가 있었음.
- \t에 대해서도 TT로 변환합니다.

In [ ]:
def replace_token(code):
  code = code.replace('\n', 'NN')
  code = code.replace('    ', 'TT')
  return code

In [ ]:
raw_code = [replace_token(i) for i in raw_code]
raw_code[:3]

['a = 24NNb = 6NNc = 8NNy = a * b // cNNprint(y)',
 'a = 8NNb = 16NNy = b // aNNprint(y)',
 'a = 12NNb = 8NNy = a * bNNprint(y)']

### 토큰화
- sentence piece를 사용하여 단어사전 및 토크나이저를 생성하고, 토큰화를 진행합니다.

In [ ]:
import sentencepiece as spm

In [ ]:
# sentencepiece를 학습시켜 토크나이저 생성 및 저장하는 함수
def generate_tokenizer(corpus,
                       vocab_size,
                       model_type='bpe',
                       lang='problem',
                       pad_id = 0,
                       bos_id = 1,
                       eos_id = 2,
                       unk_id = 3):
  input_file = f'{lang}_spm_input.txt'

  with open(input_file, 'w', encoding='utf-8') as f:
    for row in corpus:
      f.write(str(row) + '\n')

  sp_model_root = 'sentencepiece'
  if not os.path.isdir(sp_model_root): os.mkdir(sp_model_root) 

  prefix = '%s_tokenizer_%s' % (lang, model_type+str(vocab_size)) 
  prefix = os.path.join(sp_model_root, prefix) 

  spm.SentencePieceTrainer.Train('--pad_id={} --bos_id={} --eos_id={} --unk_id={} \
  --input={} --model_prefix={} --vocab_size={} --model_type={}'.format(pad_id, bos_id,\
                                                                       eos_id, unk_id, input_file,\
                                                                       prefix, vocab_size,
                                                                       model_type))

  tokenizer = spm.SentencePieceProcessor()
  tokenizer.Load(f'{prefix}.model') 

  return tokenizer

In [ ]:
# 단어사전 사이즈를 지정해줍니다. 
SRC_VOCAB_SIZE = 18312  # 문제에서 학습된 단어 모두 사용
TGT_VOCAB_SIZE = 3275   # 코드에서 학습된 단어 모두 사용

# 단어사전 생성( 문제, 코드 각각)
prob_tokenizer = generate_tokenizer(raw_prob, SRC_VOCAB_SIZE, 'bpe', 'problem')
code_tokenizer = generate_tokenizer(raw_code, TGT_VOCAB_SIZE, 'bpe', 'code')
code_tokenizer.set_encode_extra_options('bos:eos')

True

In [ ]:
# 토큰화 이전 code 확인
raw_code[3]

"from itertools import permutationsNNNNa = ['0', '9', '8', '7', '2', '1']NNy = max(list(map(''.join, permutations(a, 6))))NNprint(y)"

In [ ]:
# 토큰화 이후 code
prob_tokenizer.EncodeAsIds(raw_code[3])

[17592,
 3,
 17613,
 17592,
 3,
 18206,
 3,
 18206,
 3,
 18135,
 3,
 17592,
 3,
 17613,
 3,
 18206,
 17592,
 3,
 17613,
 3,
 18206,
 18263,
 18206,
 3,
 18263,
 497,
 17592,
 3,
 18204,
 17600,
 18204,
 17601,
 10416,
 17655,
 18204,
 17601,
 10416,
 17646,
 18204,
 17601,
 10416,
 17658,
 18204,
 17601,
 10416,
 17604,
 18204,
 17601,
 10416,
 10313,
 3,
 18265,
 497,
 205,
 18263,
 18049,
 17841,
 18135,
 3,
 18206,
 17841,
 17613,
 18263,
 3,
 17841,
 18204,
 18204,
 17596,
 3,
 17601,
 17592,
 3,
 17613,
 3,
 18206,
 18263,
 18206,
 3,
 17841,
 18263,
 17601,
 45,
 17842,
 17842,
 17842,
 17842,
 3,
 18206,
 17841,
 18265,
 17842]

### Padding
문제와 코드 각각에 대해 토큰화 및 패딩 처리

In [ ]:
from tqdm.notebook import tqdm

# 문제 및 코드를 토큰화 시켜 리스트로 저장
src_corpus = [prob_tokenizer.EncodeAsIds(raw_prob[idx]) for idx in tqdm(range(len(raw_prob)))]
tgt_corpus = [code_tokenizer.EncodeAsIds(raw_code[idx]) for idx in tqdm(range(len(raw_code)))]

# 패딩 
enc_train = tf.keras.preprocessing.sequence.pad_sequences(src_corpus,
                                                          padding='post')
dec_train = tf.keras.preprocessing.sequence.pad_sequences(tgt_corpus,
                                                          padding='post')

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

# 3- 모델 설계

### Positional encoding

In [ ]:
# postional encoding
def positional_encoding(pos, d_model):
  def cal_angle(position, i):
    return position / np.power(10000, int(i) / d_model)

  def get_posi_angle_vec(position):
    return [cal_angle(position, i) for i in range(d_model)]

  sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])
  sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
  sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])
  return sinusoid_table

### Multi-Head Attention

In [ ]:
# multi-head attention
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    self.depth = d_model // self.num_heads 

    self.W_q = tf.keras.layers.Dense(d_model) #Linear Layer
    self.W_k = tf.keras.layers.Dense(d_model)
    self.W_v = tf.keras.layers.Dense(d_model)

    self.linear = tf.keras.layers.Dense(d_model)

  
  def scaled_dot_product_attention(self, Q, K, V, mask):
    d_k = tf.cast(K.shape[-1], tf.float32) 

    QK = tf.matmul(Q, K, transpose_b =True)   # QK 내적

    scaled_qk = QK / tf.math.sqrt(d_k)   # scaled 된 QK 값 구하기

    if mask is not None: scaled_qk += (mask * -1e9)  # 후에 나올 masking 처리

    attentions = tf.nn.softmax(scaled_qk, axis=-1)  # attention weights 값 구하기
    
    out = tf.matmul(attentions, V)   # attention 값을 V에 곱하기

    return out, attentions


  def split_heads(self, x):
    """
    Embedding을 Head의 수로 분할하는 함수

    x: [ batch x length x emb ]
    return: [ batch x length x heads x self.depth ]
    """
    batch_size = x.shape[0]
    split_x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])  

    return split_x

  
  def combine_heads(self, x):
    """
    분할된 Embedding을 하나로 결합하는 함수

    x: [ batch x length x heads x self.depth ]
    return: [ batch x length x emb ]
    """
    batch_size = x.shape[0]
    combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
    combined_x = tf.reshape(combined_x, (batch_size, -1, self.d_model))

    return combined_x


  def call(self, Q, K, V, mask):
    # 1.  Linear_in(Q, K, V) -> WQ, WK, WV
    WQ = self.W_q(Q)
    WK = self.W_k(K)
    WV = self.W_v(V)

    # 2. Split Heads(WQ, WK, WV) -> WQ_split, WK_split, WV_split
    WQ_splits = self.split_heads(WQ)
    WK_splits = self.split_heads(WK)
    WV_splits = self.split_heads(WV)

    # 3. Scaled Dot Product Attention(WQ_split, WK_split, WV_split) -> out, attention_weights
    out, attention_weights = self.scaled_dot_product_attention(WQ_splits,
                                                               WK_splits,
                                                               WV_splits,
                                                               mask)
    
    # 4. Combine Heads(out) -> out
    out = self.combine_heads(out)

    # 5. Linear_out(out) -> out
    out = self.linear(out)

    return out, attention_weights

### Position wise feed forward network

In [ ]:
# position wise feed forward network
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
  def __init__(self, d_model, d_ff):
    super(PoswiseFeedForwardNet, self).__init__()
    self.w_1 = tf.keras.layers.Dense(d_ff, activation='relu')
    self.w_2 = tf.keras.layers.Dense(d_model)

  def call(self, x):
    out = self.w_1(x)
    out = self.w_2(out)

    return out

### Encoder layer

In [ ]:
## Encoder 레이어
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, n_heads, d_ff, dropout):
    super(EncoderLayer, self).__init__()

    self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
    self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

    self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout = tf.keras.layers.Dropout(dropout)


  def call(self, x, mask):
    # Multi-Head Attention
    residual = x
    out = self.norm_1(x)
    out, enc_attn = self.enc_self_attn(out, out, out, mask)
    out = self.dropout(out)
    out += residual

    # Position-Wise Feed Forward Network
    residual = out
    out = self.norm_2(out)
    out = self.ffn(out)
    out = self.dropout(out)
    out += residual

    return out, enc_attn

### Decoder layer

In [ ]:
## Decoder 레이어
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, d_ff, dropout):
    super(DecoderLayer, self).__init__()

    self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
    self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

    self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

    self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout = tf.keras.layers.Dropout(dropout)


  def call(self, x, enc_out, causality_mask, padding_mask):
    # Masked Multi-Head Attention
    residual = x
    out = self.norm_1(x)
    out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
    out = self.dropout(out)
    out += residual

    # Multi-Head Attention
    residual = out
    out = self.norm_2(out)
    out, dec_enc_attn = self.enc_dec_attn(out, enc_out, enc_out, causality_mask)
    out = self.dropout(out)
    out += residual

    # Position-Wise Feed Forward Network
    residual = out
    out = self.norm_3(out)
    out = self.ffn(out)
    out = self.dropout(out)
    out += residual

    return out, dec_attn, dec_enc_attn

### Encoder class

In [ ]:
## Encoder class
class Encoder(tf.keras.Model):
  def __init__(self,
               n_layers,
               d_model,
               n_heads,
               d_ff,
               dropout):
    super(Encoder, self).__init__()
    self.n_layers = n_layers
    self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]


  def call(self, x, mask):
    out = x

    enc_attns = list()
    for i in range(self.n_layers):
      out, enc_attn = self.enc_layers[i](out, mask)
      enc_attns.append(enc_attn)

    return out, enc_attns

### Decoder class

In [ ]:
## Decoder class
class Decoder(tf.keras.Model):
  def __init__(self,
               n_layers,
               d_model,
               n_heads,
               d_ff,
               dropout):
    super(Decoder, self).__init__()
    self.n_layers = n_layers
    self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]


  def call(self, x, enc_out, causality_mask, padding_mask):
    out = x
    
    dec_attns = list()
    dec_enc_attns = list()
    for i in range(self.n_layers):
      out, dec_attn, dec_enc_attn = self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

      dec_attns.append(dec_attn)
      dec_enc_attns.append(dec_enc_attn)

    return out, dec_attns, dec_enc_attns

### Transformer

In [ ]:
# Transformer
class Transformer(tf.keras.Model):
  def __init__(self,
               n_layers,
               d_model,
               n_heads,
               d_ff,
               src_vocab_size,
               tgt_vocab_size,
               pos_len,
               dropout=0.2,
               shared=True):
    super(Transformer, self).__init__()
    self.d_model = tf.cast(d_model, tf.float32)
    
    # 1. Embedding Layer 정의
    self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
    self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

    # 2. Positional Encoding 정의
    self.pos_encoding = positional_encoding(pos_len, d_model)

    # 3. Dropout 정의
    self.dropout = tf.keras.layers.Dropout(dropout) 

    # 4. Encoder / Decoder 정의
    self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
    self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)
    
    # 5. Output Linear 정의
    self.fc = tf.keras.layers.Dense(tgt_vocab_size)

    # 6. Shared Weights
    self.shared = shared

    if shared: self.fc.set_weights(tf.transpose(self.dec_emb.weights))


  def embedding(self, emb, x):
    """
    입력된 정수 배열을 Embedding + Pos Encoding
    + Shared 일 경우 Scaling 작업 포함

    x: [batch x length]
    return: [batch x length x emb]
    """
    seq_len = x.shape[1]
    out = emb(x)

    if self.shared: out *= tf.math.sqrt(self.d_model)

    out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
    out = self.dropout(out)

    return out


  def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
    """
    아래 순서에 따라 작성

    Step 1: Embedding(enc_in, dec_in) -> enc_in, dec_in
    Step 2: Encoder(enc_in, enc_mask) -> enc_out, enc_attns
    Step 3: Decoder(dec_in, enc_out, mask) -> dec_out, dec_attns, dec_enc_attns
    Step 4: Out Linear(dec_out) -> logits
    """
    enc_in = self.embedding(self.enc_emb, enc_in)
    dec_in = self.embedding(self.dec_emb, dec_in)

    enc_out, enc_attns = self.encoder(enc_in, enc_mask)

    dec_out, dec_attns, dec_enc_attns = \
    self.decoder(dec_in, enc_out, causality_mask, dec_mask)

    logits = self.fc(dec_out)

    return logits, enc_attns, dec_attns, dec_enc_attns

# 4- 훈련하기

#### transformer 선언

In [ ]:
transformer = Transformer(n_layers=6,
                          d_model=512,
                          n_heads=8,
                          d_ff=2048,
                          src_vocab_size=SRC_VOCAB_SIZE,
                          tgt_vocab_size=TGT_VOCAB_SIZE,
                          pos_len=100,
                          dropout=0.2,
                          shared=True)

#### learning rate scheduler 및 adam optimizer 선언

In [ ]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps = 4000):
    super(LearningRateScheduler, self).__init__()
    self.d_model = d_model
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = step ** -0.5
    arg2 = step * (self.warmup_steps ** -1.5)

    return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)


learning_rate = LearningRateScheduler(512)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                     beta_1 = 0.9,
                                     beta_2 = 0.98,
                                     epsilon = 1e-9)


#### loss 함수 정의

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits = True, reduction='none'
)

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  #Masking 되지 않은 입력의 개수로 Scaling 하는 과정
  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

#### masking 정의

In [ ]:
def generate_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32) 
  return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
  mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
  return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
  enc_mask = generate_padding_mask(src)
  dec_mask = generate_padding_mask(tgt)

  dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
  dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

  dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
  dec_mask = tf.maximum(dec_mask, dec_causality_mask)

  return enc_mask, dec_enc_mask, dec_mask


#### train step 함수 정의

In [ ]:
@tf.function()
def train_step(src, tgt, model, optimizer):
  gold = tgt[:, 1:]

  enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt)

  # 계산된 loss에 tf.GradientTape()를 적용해 학습을 진행한다.
  with tf.GradientTape() as tape:
    predictions, enc_attns, dec_attns, dec_enc_attns = model(src, tgt, enc_mask, dec_enc_mask, dec_mask)

    loss = loss_function(gold, predictions[:, :-1])

    # 최종적으로 optimizer.apply_gradients()가 사용됩니다.
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

#### 코드 생성 함수

In [ ]:
# 코드 생성 함수
def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):

  pieces = src_tokenizer.encode_as_pieces(sentence)
  tokens = src_tokenizer.encode_as_ids(sentence)

  _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                         maxlen=enc_train.shape[-1],
                                                         padding = 'post')
  ids = []
  output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)
  for i in range(dec_train.shape[-1]):
    enc_padding_mask, combined_mask, dec_padding_mask = \
    generate_masks(_input, output)

    predictions, enc_attns, dec_attns, dec_enc_attns = \
    model(_input,
          output,
          enc_padding_mask,
          combined_mask,
          dec_padding_mask)
    
    predicted_id = \
    tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

    if tgt_tokenizer.eos_id() == predicted_id:
      result = tgt_tokenizer.decode_ids(ids)
      return pieces, result, enc_attns, dec_attns, dec_enc_attns

    ids.append(predicted_id)
    output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

  result = tgt_tokenizer.decode_ids(ids)

  return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [ ]:
# 코드 생성
def translate(sentence, model, src_tokenizer, tgt_tokenizer):
  pieces, result, enc_attns, dec_attns, dec_enc_attns = \
  evaluate(sentence, model, src_tokenizer, tgt_tokenizer)

  #print('Input: %s' % (sentence))
  #print('Predicted translation: {}'.format(result))

  return result

#### 학습 진행
- early stopping은 넣지 않았습니다.

In [ ]:
from tqdm import tqdm_notebook

BATCH_SIZE = 64
EPOCHS = 50

for epoch in range(EPOCHS):
  total_loss = 0

  idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
  random.shuffle(idx_list)
  t = tqdm_notebook(idx_list)

  for (batch, idx) in enumerate(t):
    batch_loss, enc_attns, dec_attns, dec_enc_attns = \
    train_step(enc_train[idx:idx+BATCH_SIZE],
               dec_train[idx:idx+BATCH_SIZE],
               transformer,
               optimizer)
    
    total_loss += batch_loss

    t.set_description_str('Epoch %2d' % (epoch + 1))
    t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

...

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

# 5- 모델 저장

In [ ]:
transformer.save_weights('/content/drive/MyDrive/Ko-Co/transformer_basecode.h5')

In [ ]:
#transformer.load_weights('/content/drive/MyDrive/Ko-Co/transformer_basecode.h5')

# 6- 답 생성하기

#### 답 생성 코드

In [ ]:
# 생성된 코드로부터 답을 계산하는 함수
def exec_code(codes:list) -> list:
    """ 성공하면 print 값을, 실패하면 fail을 반환하는 함수 
        print 되는 값을 test_file에 저장 후 다시 불러와 
        후처리를 한 후에 리스트로 반환합니다
    """

    import sys
    f = open('test_ans.txt', 'w')
    sys.stdout = f
    for i in codes:
        try:
            exec(i)
        except:
            print('fail')
    sys.stdout = sys.__stdout__
    f.close()
    with open('test_ans.txt', 'r') as f:
        txt = f.readlines()
    test_text = [i.replace('\n', "") for i in txt]
    return test_text

### 예시 코드로 확인해보기

In [ ]:
examples = [
            '둘레가 같은 정육각형과 정팔각형이 있습니다. 정팔각형의 한 변의 길이가 12cm일 때, 정육각형의 한 변의 길이는 몇 cm인지 구해보세요.',
	          '80과 120을 어떤 수로 나누면 두 수 모두 나누어떨어집니다. 어떤 수 중에서 가장 큰 수를 구해 보시오.',
          	'들이가 8L인 세숫대야에 물이 반만큼 들어 있습니다. 이 중에서 세수를 하는 데 2900mL를 사용했습니다. 남은 물의 양은 몇 mL인가요?',
            '정민이가 20일 동안 12000원을 모으려고 할 때, 하루에 얼마씩 모아야 하나요?',
            '수 카드 8, 2, 5, 6, 9이 있습니다. 수 카드 중 4장을 뽑아 한 번씩만 사용하여 세 자리 수를 만들려고 합니다. 만들 수 있는 가장 큰 수와 가장 작은 수의 차는 얼마가 될까?'
]

In [ ]:
# 위 예시를 코드로 번역하여 저장
codes = [translate(example, transformer, prob_tokenizer, code_tokenizer) for example in examples]

In [ ]:
# 생성된 코드 확인
codes

['a = 6NNb = 8NNc = 1NNy = a * b // cNNprint(y)',
 'a = 120NNb = 1000NNc = 180NNy = a + b // cNNprint(y)',
 'a = 8NNb = 0.7NNc = 20NNy = a - (b * c)NNprint(y)',
 'a = 20NNb = 9NNc = 5NNy = (a * b) // c)NNprint(y)',
 "from itertools import permutationsNNNNa = ['2', '5', '7', '8', '9']NNb = 3NNc = list(map(''.join, permutations(a, b))))NNd = int(max(c))NNy = d + eNNprint(y)"]

### 전처리 함수

In [ ]:
## NN 을 \n으로 다시 바꿔주는 작업
def alt_NN(codes):
  codes = codes.replace('NN', '\n')
  codes = codes.replace('TT', '\t')
  return codes

In [ ]:
codes = [alt_NN(code) for code in codes]
codes 

['a = 6\nb = 8\nc = 1\ny = a * b // c\nprint(y)',
 'a = 120\nb = 1000\nc = 180\ny = a + b // c\nprint(y)',
 'a = 8\nb = 0.7\nc = 20\ny = a - (b * c)\nprint(y)',
 'a = 20\nb = 9\nc = 5\ny = (a * b) // c)\nprint(y)',
 "from itertools import permutations\n\na = ['2', '5', '7', '8', '9']\nb = 3\nc = list(map(''.join, permutations(a, b))))\nd = int(max(c))\ny = d + e\nprint(y)"]

In [ ]:
# 생성된 코드 실행해서 답 확인하기
exec_code(codes)

['48', '125', '-6.0', 'fail', 'fail']

# 7- json 파일로 변환하기

### test 데이터 불러오기

In [ ]:
import pandas as pd
test_path = '/content/drive/MyDrive/Ko-Co/KMWP/data/test.csv'

test_data = pd.read_csv(test_path)
test_data.head()

,idx,problem
0,1,둘레가 같은 정육각형과 정팔각형이 있습니다. 정팔각형의 한 변의 길이가 12cm일 ...
1,2,80과 120을 어떤 수로 나누면 두 수 모두 나누어떨어집니다. 어떤 수 중에서 가...
2,3,들이가 8L인 세숫대야에 물이 반만큼 들어 있습니다. 이 중에서 세수를 하는 데 2...
3,4,"정민이가 20일 동안 12000원을 모으려고 할 때, 하루에 얼마씩 모아야 하나요?"
4,5,"수 카드 8, 2, 5, 6, 9이 있습니다. 수 카드 중 4장을 뽑아 한 번씩만 ..."


In [ ]:
# 문제를 리스트로 바꿔줍니다
test_prob = test_data.problem.to_list()
test_prob[:3]

['둘레가 같은 정육각형과 정팔각형이 있습니다. 정팔각형의 한 변의 길이가 12cm일 때, 정육각형의 한 변의 길이는 몇 cm인지 구해보세요.',
 '80과 120을 어떤 수로 나누면 두 수 모두 나누어떨어집니다. 어떤 수 중에서 가장 큰 수를 구해 보시오.',
 '들이가 8L인 세숫대야에 물이 반만큼 들어 있습니다. 이 중에서 세수를 하는 데 2900mL를 사용했습니다. 남은 물의 양은 몇 mL인가요?']

### 코드 및 답 생성 함수

In [ ]:
def generate_code_and_ans(test_prob):
  codes = [translate(example, transformer, prob_tokenizer, code_tokenizer) for example in test_prob]
  codes = [alt_NN(code) for code in codes]
  answers = exec_code(codes)
  return codes, answers

In [ ]:
# 코드와 답을 생성해 저장합니다.
## 20분 정도 걸림

codes, answers = generate_code_and_ans(test_prob)

In [ ]:
codes[:3]

['a = 6\nb = 8\nc = 1\ny = a * b // c\nprint(y)',
 'a = 120\nb = 1000\nc = 180\ny = a + b // c\nprint(y)',
 'a = 8\nb = 0.7\nc = 20\ny = a - (b * c)\nprint(y)']

In [ ]:
answers[:3]

['48', '125', '-6.0']

In [ ]:
len(test_prob)

282

In [ ]:
len(codes), len(answers)

(282, 282)

In [ ]:
# 답이 안 나온 문제 개수 확인
answers.count('fail')

87

### json 파일 생성
- Dataframe -> json 파일로 변환

In [ ]:
submit_data = test_data.copy()

submit_data.head()

,idx,problem
0,1,둘레가 같은 정육각형과 정팔각형이 있습니다. 정팔각형의 한 변의 길이가 12cm일 ...
1,2,80과 120을 어떤 수로 나누면 두 수 모두 나누어떨어집니다. 어떤 수 중에서 가...
2,3,들이가 8L인 세숫대야에 물이 반만큼 들어 있습니다. 이 중에서 세수를 하는 데 2...
3,4,"정민이가 20일 동안 12000원을 모으려고 할 때, 하루에 얼마씩 모아야 하나요?"
4,5,"수 카드 8, 2, 5, 6, 9이 있습니다. 수 카드 중 4장을 뽑아 한 번씩만 ..."


In [ ]:
# submit_data에 생성된 code와 answer를 저장합니다
submit_data['code'] = codes
submit_data['answer'] = answers

submit_data.head()

,idx,problem,code,answer
0,1,둘레가 같은 정육각형과 정팔각형이 있습니다. 정팔각형의 한 변의 길이가 12cm일 ...,a = 6\nb = 8\nc = 1\ny = a * b // c\nprint(y),48
1,2,80과 120을 어떤 수로 나누면 두 수 모두 나누어떨어집니다. 어떤 수 중에서 가...,a = 120\nb = 1000\nc = 180\ny = a + b // c\npr...,125
2,3,들이가 8L인 세숫대야에 물이 반만큼 들어 있습니다. 이 중에서 세수를 하는 데 2...,a = 8\nb = 0.7\nc = 20\ny = a - (b * c)\nprint(y),-6.0
3,4,"정민이가 20일 동안 12000원을 모으려고 할 때, 하루에 얼마씩 모아야 하나요?",a = 20\nb = 9\nc = 5\ny = (a * b) // c)\nprint(y),fail
4,5,"수 카드 8, 2, 5, 6, 9이 있습니다. 수 카드 중 4장을 뽑아 한 번씩만 ...",from itertools import permutations\n\na = ['2'...,fail


In [ ]:
submit_data.set_index('idx', inplace=True)
submit_data.head()

,problem,code,answer
idx,,,
1,둘레가 같은 정육각형과 정팔각형이 있습니다. 정팔각형의 한 변의 길이가 12cm일 ...,a = 6\nb = 8\nc = 1\ny = a * b // c\nprint(y),48
2,80과 120을 어떤 수로 나누면 두 수 모두 나누어떨어집니다. 어떤 수 중에서 가...,a = 120\nb = 1000\nc = 180\ny = a + b // c\npr...,125
3,들이가 8L인 세숫대야에 물이 반만큼 들어 있습니다. 이 중에서 세수를 하는 데 2...,a = 8\nb = 0.7\nc = 20\ny = a - (b * c)\nprint(y),-6.0
4,"정민이가 20일 동안 12000원을 모으려고 할 때, 하루에 얼마씩 모아야 하나요?",a = 20\nb = 9\nc = 5\ny = (a * b) // c)\nprint(y),fail
5,"수 카드 8, 2, 5, 6, 9이 있습니다. 수 카드 중 4장을 뽑아 한 번씩만 ...",from itertools import permutations\n\na = ['2'...,fail


In [ ]:
# dataframe을 json으로 바로 변환
submit_data_path = '/content/drive/MyDrive/Ko-Co/KMWP/data/transformer_submit.json'
submit_data.to_json(submit_data_path, orient='index', indent=4)